In [243]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys  # Импортируем Keys

import time
driver = webdriver.Safari() 
driver.get("https://www.avito.ru/moskva/rabota") 


In [244]:
element = driver.find_element(By.ID, "bx_search") 
element.send_keys('Уборщик') 
element.send_keys(Keys.ENTER)


In [245]:
button = driver.find_element(By.CSS_SELECTOR, "a[data-marker='category[1000061]/clickable']")
button.click()

In [250]:
from selenium.webdriver.common.action_chains import ActionChains

actions = ActionChains(driver)
actions.scroll_by_amount(delta_x=-50, delta_y=+500).perform()
